In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd

additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

cbr_database = cbr_database.set_index("id")

EX_APP = "f"

INITIAL_C = 3
GENERATED_C = 12

DATABASE_NAME = f"cbr_database_{INITIAL_C}_{GENERATED_C}_{EX_APP}"
OUTPUT_FILE = f"cbr_{INITIAL_C}_{GENERATED_C}_{EX_APP}_gpt2_input"

IX_FILE = "cbr_augmentation_12_gpt2_results_p_ix_f.csv"
IX_FILE2 = "cbr_augmentation_gpt2_results_p_ix_f.csv"
dfix1 = pd.read_csv(f"{IX_FILE}")
dfix2 = pd.read_csv(f"{IX_FILE2}")

dfix = pd.concat([dfix1, dfix2])
print(len(dfix))


17835


In [2]:
# cbr_db = pd.concat([cbr_database, additional_cbr_1])
cbr_database = cbr_database[:int(INITIAL_C*1000)]
dfix = dfix[:int(GENERATED_C*1000)]
dfix = dfix.set_index("id")
cbr_db_ix = pd.concat([cbr_database, dfix])
# cbr_db_ix = cbr_db_ix.set_index("id")

cbr_reps = str_rep_vdb(cbr_db_ix)
len(cbr_reps)

15000

In [3]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db_ix.index.values)], {"dataset":DATABASE_NAME})
vdb = CBRDatabase(cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

# vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)

print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=2))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
15000
[Document(page_content='name: easy no fail chocolate fudge no thermometer ; ingredients: sugar, evaporated milk, butter, salt, mini marshmallows, semi-sweet chocolate chips, walnuts, vanilla ; preparation:', metadata={'dataset': 'cbr_database_3_12_f', 'recipe_id': 80447}), Document(page_content='name: weight watcher 1 point banana bread flex points ; ingredients: bananas, splenda sugar substitute, natural applesauce, all-purpose flour, whole wheat flour, salt, baking soda ; preparation:', metadata={'dataset': 'cbr_database_3_12_f', 'recipe_id': 154636})]


In [4]:
# tem = vdb.similarity_search(query="akjsjd", k=5)
# recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
# recipe_ixs

In [5]:
# generowanie inputów - 850 tokenów 
from pprint import pprint
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        
        #dev
        # print(tem)
        # composed = compose(cbr_db_ix.loc[recipe_ixs[0:2]], v_recipe)
        # print(recipe_ixs)
        # print(composed)
        # pprint(composed.split("name: "))
        # raise
        #dev

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 794
1 - 684
1 - 608
1 - 747
1 - 796
1 - 695
1 - 717
1 - 850
1 - 670
1 - 720
1 - 754
1 - 837
1 - 810
2 - 660
1 - 835
1 - 712
1 - 787
1 - 836
1 - 679
1 - 671
1 - 723
1 - 655
2 - 632
1 - 753
1 - 726
1 - 749
1 - 756
1 - 798
1 - 782
1 - 606
1 - 719
1 - 808
1 - 747
1 - 629
1 - 777
1 - 746
1 - 720
1 - 741
1 - 716
1 - 655
1 - 682
1 - 624
1 - 800
1 - 804
1 - 733
1 - 778
1 - 714
1 - 716
1 - 748
1 - 720
1 - 772
1 - 829
1 - 846
1 - 657
1 - 846
1 - 722
1 - 810
1 - 765
1 - 814
1 - 807
1 - 777
1 - 806
1 - 786
1 - 693
1 - 744
1 - 735
1 - 673
1 - 669
1 - 783
1 - 798
1 - 660
2 - 654
2 - 765
1 - 680
1 - 757
1 - 664
1 - 700
1 - 704
1 - 749
1 - 735
1 - 824
1 - 742
1 - 716
1 - 724
1 - 787
1 - 776
1 - 701
1 - 645
1 - 747
1 - 655
1 - 724
1 - 768
1 - 715
1 - 812
1 - 839
1 - 813
2 - 792
1 - 780
1 - 781
1 - 849
1 - 730
1 - 740
1 - 763
1 - 719
1 - 784
1 - 825
1 - 748
1 - 764
1 - 747
1 - 697
1 - 704
1 - 744
1 - 699
1 - 735
1 - 780
1 - 694
1 - 710
1 - 835
1 - 770
1 - 805
1 - 784
1 - 797
1 - 767
1 - 835
1 - 690


In [6]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [7]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=4)

[Document(page_content='name: v s do ahead slow cooker mashed potatoes ; ingredients: potatoes, water, butter, salt, garlic powder, ground black pepper, milk ; preparation:', metadata={'dataset': 'cbr_database_3_12_f', 'recipe_id': 196153}),
 Document(page_content='name: naked tater all the way ; ingredients: potatoes, butter, sour cream, milk, bacon bits, cheddar cheese, green onions, garlic powder, salt and pepper ; preparation:', metadata={'dataset': 'cbr_database_3_12_f', 'recipe_id': 135691}),
 Document(page_content='name: roasted garlic make ahead mashed potatoes ; ingredients: potatoes, chicken broth, cream cheese, roasted garlic clove, salt and pepper, parmesan cheese ; preparation:', metadata={'dataset': 'cbr_database_3_12_f', 'recipe_id': 232332}),
 Document(page_content='name: potatoes stroganoff ; ingredients: lean ground beef, garlic powder, betty crocker au gratin potatoes, hot water, milk, green giant mushroom pieces and stems, sour cream ; preparation:', metadata={'data

In [8]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1125, 7: 965, 5: 35})